# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.

In [130]:
"""Definición de polinomios de Taylor, donde las entradas del tipo son los coeficientes normalizados de Taylor 
de una función dada alrededor de un mismo punto.
...
"""
type Taylor{D<:Number}
    coef :: Array{D,1}
    order :: Int
    
    #Definimos una función que manipule el orden de un polinomio de taylor
    function Taylor(coef :: Array{D,1}, order :: Int)
        l = length(coef) #Definimos la longitud del Taylor
        order = max(order,l-1) #Vemos que valor es más grande
        order == l-1 && return new(coef, order) #si el arreglo era mas grande que el orden dado, genera uno nuevo
        resize!(coef, order+1) #cambia la longitud del arreglo a order+1 (empieza a contar en 1 y el orden en 0)
        for i = l+1:order+1 #Agrega los ceros necesarios para completar el taylor
            coef[i]=zero(D)
        end
        new(coef, order) #Regresa el nuevo Taylor
    end
    
end

Taylor{D<:Number}(x::Taylor{D}, order::Int) = Taylor{D}(x.coef, order) # El taylor de un taylor es si mismo
Taylor{D<:Number}(x::Taylor{D}) = x #lo mismo que arriba pero sin especificar el orden
Taylor{D<:Number}(coef::Array{D,1}, order::Int) = Taylor{D}(coef, order) # Permite generar taylors a partir de un arreglo y un orden
Taylor{D<:Number}(coef::Array{D,1}) = Taylor{D}(coef, length(coef)-1) # Lo mismo pero sin especificar el orden
Taylor{D<:Number}(x::D, order::Int) = Taylor{D}([x], order) # Taylor de una constante con orden definido
Taylor{D<:Number}(x::D) = Taylor{D}([x], 0) #Taylor de una constante

LoadError: LoadError: invalid redefinition of constant Taylor
while loading In[130], in expression starting on line 442

In [131]:
Taylor(a)=Taylor(promote(a)...)

Taylor{D<:Number}

In [132]:
import Base: +, -, *, /, ==

#Definimos con el mínimo para no tener que agregar ceros. De por sí por como definimos Taylor manualmente puedes especificar 
# el orden necesario.
function +(a::Taylor,b::Taylor)
    m = minimum([a.order,b.order])+1
    c = zeros(m)
    for k in range(1,m)
        c[k]=a.coef[k]+b.coef[k]
    end
    return Taylor(c)
end

function -(a::Taylor,b::Taylor)
    m = minimum([a.order,b.order])+1
    c = zeros(m)
    for k in range(1,m)
        c[k]=a.coef[k]-b.coef[k]
    end
    return Taylor(c)
end

function *(a::Taylor,b::Taylor)
    m = minimum([a.order,b.order])+1
    c = zeros(m)
    for k in range (1,m)
        for i in range(1,k)
            c[k]+=a.coef[i]*b.coef[k-i+1]
        end
    end
    return Taylor(c)
end

function /(a::Taylor,b::Taylor)
    m = minimum([a.order,b.order])+1
    c = zeros(m)
    for k in range (1,m)
        s = 0
        for i in range(1,k-1)
            s += c[i]*b.coef[k-i+1]
        end
        c[k] = (a.coef[k]-s)/b.coef[1]
    end
    return Taylor(c)
end

function /(a::Taylor,b::Taylor)
    for k in range (1,b.order+1)
        global l
        if b.coef[k] != 0
            l=k
            break
        end
    end
    m = maximum([a.order,l])
    c = zeros(m)
    a = Taylor(a.coef,m)
    b = Taylor(b.coef,m)
    for k in range(1,m-l+1)
        s = 0
        for i in range(1,k-l)
            s += c[i]*b.coef[k+l-i]
        end
        c[k] = (a.coef[k+l-1]-s)/b.coef[l]
    end
    return Taylor(c)
end

function ==(a::Taylor,b::Taylor)
    m = maximum([a.order,b.order])
    #Agregamos ceros
    a = Taylor(a.coef,m)
    b = Taylor(b.coef,m)
    for k in range(1,m+1)
        if a.coef[k] != b.coef[k]
            return false
        end
    end
    return true
end

# Aqui se implementan los métodos necesarios para cada función

== (generic function with 110 methods)

In [133]:
T=Taylor([2,2],3)


Taylor{Int64}([2,2,0,0],3)

In [134]:
Taylor(T.coef,4)

Taylor{Int64}([2,2,0,0,0],4)

In [135]:
Taylor([0,1,1])/Taylor([0,0,2])

Taylor{Float64}([0.5,0.0,0.0],2)

In [136]:
Taylor([4,5,6])

Taylor{Int64}([4,5,6],2)

In [137]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test


# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

Función Logaritmo

Se toma $$L(x)=log(g(x))$$

Al calcular la derivada se obtiene

\begin{equation}
\frac{{\rm d} L(x)}{{\rm d}x} = \frac{g'(x)}{g(x)}
\end{equation}

Además se define \begin{equation}
L(x) = \sum_{k=0}^\infty L_{[k]} (x-x_0)^{k}
\end{equation}

Entonces \begin{equation}
\frac{{\rm d} L(x)}{{\rm d}x} = \sum_{k=1}^\infty k\ L_{[k]}\, (x-x_0)^{k-1}
\end{equation}

Además \begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^{k}
\end{equation}

Lo que implica que \begin{equation}
g'(x)=\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k\ g_{[k]}\, (x-x_0)^{k-1} 
\end{equation}

Por otro lado, partiendo de que \begin{equation} \Big(\frac{f}{g}\Big)_{[k]}  =  \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) \end{equation}


Se toma \begin{equation}
\frac{f}{g} \rightarrow \frac{g'}{g}
\end{equation}

Por tanto, se llega a que 

\begin{equation} \Big(\frac{g'}{g}\Big)_{[k]}  =  \frac{1}{g_{[0]}}
\Big( g'_{[k]} - \sum_{i=0}^{k-1} \big(\frac{g'}{g}\big)_{[i]} \, g_{[k-i]} \Big) 
\end{equation}

\begin{equation}
\Rightarrow k\ L_{[k]}\ = \frac{1}{g_{[0]}}
\Big(k\ g_{[k]}\ - \sum_{i=0}^{k-1} i\ L_{[i]} \ g_{[k-i]}   \Big) 
\end{equation}

\begin{equation}
\Rightarrow L_{[k]}\ = \frac{1}{k\ g_{[0]}}
\Big(k\ g_{[k]}\ - \sum_{i=0}^{k-1} i\ L_{[i]} \ g_{[k-i]}   \Big) 
\end{equation}

In [138]:
function taylog(x0,a::Taylor)
    m = (a.order+1)
    L = zeros(m)
    for k in range(1,m)
        global l
        if a.coef[k] != 0
            l=k
            break
        end
    end
    #L[1] = log(x0)
    for k in range(1,m-l+1)
        s = 0
        for i in range(1,k-l)
            s += i*L[i]*a.coef[k+l-i]
        end
        L[k] = ((k+l-1)a.coef[k+l-1]-s)/((k-l+1)*a.coef[l])
    end
    return Taylor(L)
end    


taylog (generic function with 3 methods)

In [139]:
taylog(1,Taylor([1,1],5))

Taylor{Float64}([1.0,0.5,-0.3333333333333333,0.25,-0.2,0.16666666666666666],5)

Función $P_\alpha(x) = \big(g(x)\big)^\alpha$

Al calcular la derivada se obtiene

\begin{equation}
\frac{{\rm d} P_\alpha(x)}{{\rm d}x} = \alpha \ \big(g(x)\big)^{\alpha -1}
\end{equation}

\begin{equation}
\Rightarrow \frac{{\rm d} P_\alpha(x)}{{\rm d}x} = \frac{\alpha \ P_\alpha(x) \ g'(x) }{g(x)}
\end{equation}

Además se define \begin{equation}
P_\alpha(x) = \sum_{k=0}^\infty P_{\alpha [k]} (x-x_0)^{k}
\end{equation}

Entonces \begin{equation}
\frac{{\rm d} P_\alpha}{{\rm d}x} = \sum_{k=1}^\infty k\ P_{\alpha[k]}\, (x-x_0)^{k-1}
\end{equation}

Además \begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^{k}
\end{equation}

Lo que implica que \begin{equation}
g'(x)=\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k\ g_{[k]}\, (x-x_0)^{k-1} 
\end{equation}

Por otro lado, partiendo de que \begin{equation} \Big(\frac{f}{g}\Big)_{[k]}  =  \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) \end{equation}


Se toma \begin{equation}
\frac{f}{g} \rightarrow \frac{\alpha \ P_\alpha(x) \ g'(x) }{g(x)}
\end{equation}

Por tanto, se llega a que 

\begin{equation} \Big(\frac{\alpha \ P_\alpha(x) \ g'(x)}{g}\Big)_{[k]}  =  \frac{1}{g_{[0]}}
\Big( \big(\alpha \ P_\alpha(x) \ g'(x)\big)_{[k]} - \sum_{i=0}^{k-1} \big(\frac{\alpha \ P_\alpha(x) \ g'(x)}{g}\big)_{[i]} \, g_{[k-i]} \Big) 
\end{equation}

\begin{equation}
\Rightarrow k\ P_\alpha(x)\ = \frac{1}{g_{[0]}}
\Big(\alpha \ \sum_{i=1}^{k} j\ g_{[j]}\ P_{\alpha [k-j]}\ - \sum_{i=1}^{k-1} i\ P_{\alpha(x)[i]} \ g_{[k-i]}   \Big) 
\end{equation}

\begin{equation}
\Rightarrow P_\alpha(x)\ = \frac{1}{k \ g_{[0]}}
\Big(\alpha \ \sum_{i=1}^{k} j\ g_{[j]}\ P_{\alpha [k-j]}\ - \sum_{i=1}^{k-1} i\ P_{\alpha(x)[i]} \ g_{[k-i]}   \Big)
\end{equation}

In [140]:
function taypot(α::Int,a::Taylor)
    for k in range(1,α)
        global pot
        pot *= a
    end
    return pot
end
    

taypot (generic function with 1 method)